In [ ]:
# TODO UPDATE THIS OLD MESSY CODE

In [1]:
import pandas as pd
import numpy as np
import itertools
import time
import tensorflow as tf
import pickle
import sklearn as sk
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from keras.callbacks import EarlyStopping
from tensorflow.keras import initializers
from keras.models import Model
from dateutil.relativedelta import relativedelta
from datetime import datetime
from typing import Dict, List

In [2]:
# TEMP TO RUN ON CPU
tf.config.set_visible_devices([], 'GPU')

In [3]:
def dropRowsAndColsForCA(df):
    # Cut to characteristics columns of interest
    df = df[['date', 'asset', 'r_tplus1', ]] # TODO GET ~50-100 CHARACTERISTICS THAT OWRK BEST IN UNI WORK?

    # TODO NEED TO FIGURE OUT WHERE WE HAVE 2x NUMBER OF ASSETS AS CHARACTERISTICS
    
    # Cut panel to the first week where there are twice as many tokens as RHS vars
    df['counts'] = 1
    df['coins_per_week'] = df.groupby(['date'])['counts'].transform(sum)
    df = df[df.coins_per_week >= (df.shape[1]-2)*2]
    df = df.drop(columns = ['counts', 'coins_per_week'])

    # Note: keep obs to RHS ratio roughly 4e4:1

    # Note: for any macro, take cartesian product iwth characteristics to make it characteritisc level
    # -or do teh reg thing to reduce it down to same dim as number of assets

    return df

In [4]:
def formPortfolioReturnCovariates(df):
    # Obtain the datetimes of the dataframe
    df = df.sort_values(by = 'date')
    weeks = np.unique(df.index)

    # Form new covariate names
    column_names = list(df.columns.values)
    column_names.remove('asset')
    column_names.remove('r_tplus7')
    covariates = column_names
    new_covariates = ['x_' + cov for cov in covariates]

    # TODO ADD A CONSTANT TO z_t before reg?

    for current_week in weeks: 
        # Obtain the week's LHS returns and the previous week's covariates
        r_tp1 = df[df.index == current_week].r_tplus1.values
        z_t  = df[df.index == current_week][covariates].values

        # Calculate the characteristic managed portfolio returns
        design = np.linalg.inv(np.matmul(np.transpose(z_t), z_t))
        x_tp1    = np.matmul(np.matmul(design, np.transpose(z_t)), r_tp1)

        # Set the new columns to this week's vector's value
        df.loc[df.index == current_week, new_covariates] = x_tp1

    return df


In [5]:
def processMonth(panel_df: pd.DataFrame, date_key: str, asset_list: List[str]) -> pd.DataFrame:
    """ Process a single month's data and return the resulting DataFrame.
    """
    # Convert the date_key string to a datetime object to work with pandas
    date_key_dt = pd.to_datetime(date_key)

    # Create a date mask for the month
    date_mask = (panel_df['date'].dt.year == date_key_dt.year) & (panel_df['date'].dt.month == date_key_dt.month)

    # Subset the panel_df DataFrame based on the date mask and the asset list
    subset = panel_df[date_mask & panel_df['asset'].isin(asset_list)]

    return subset

def subsetByMonthToAssetUniverse(panel_df: pd.DataFrame, asset_universe_dict: Dict[str, List[str]], n_jobs=-1) -> pd.DataFrame:
    """
    Subset the panel data to the assets in each month (key) of asset_universe_dict.

    Args:
        panel_df: A Pandas DataFrame containing panel data at the asset-hour level
                    with ID columns 'date' and 'asset'.
        asset_universe_dict: A dictionary with keys as dates in the format YYYY-MM-DD 
                                and values as lists of asset strings.
        n_jobs: Number of CPU cores to use for parallelization. Default is -1, which means using all available cores.
    
    Returns: A Pandas DataFrame without the rows not included in the study.
    """
    # Run the process_month function in parallel using joblib
    results = Parallel(n_jobs=n_jobs)(delayed(processMonth)(panel_df, date_key, asset_list) for date_key, asset_list in asset_universe_dict.items())

    # Combine the results into a single DataFrame
    new_df = pd.concat(results, ignore_index=True)

    return new_df


In [7]:
def fitAutoencoder(train_df, hps_yhats_dict, val_df=None, early_stopping=True):
    # Obtain the covariates
    # TODO REPLACE WITH PASSING IN LHS AND RHS NAMES
    column_names = list(df.columns.values)
    column_names.remove('week_idx')
    column_names.remove('asset')
    column_names.remove('r_tplus7')
    covariates = column_names

    # Obtain the covariates on x_t and b_t_1 sides of the network
    # TODO EDIT DIS
    x_t   = [covar for covar in covariates if covar[:2] == 'x_']
    b_t_1 = [covar for covar in covariates if covar[:2] != 'x_']

    # Extract the hyperparameters
    number_hidden_layer = hps_yhats_dict['number_hidden_layer']
    number_factor       = hps_yhats_dict['number_factor']
    learning_rate       = hps_yhats_dict['learning_rate']
    l1_penalty          = hps_yhats_dict['l1_penalty']
    batch_size          = hps_yhats_dict['batch_size']
    number_ensemble     = hps_yhats_dict['number_ensemble']
    bootstrap_pct       = hps_yhats_dict['bootstrap_pct']
    epoch               = hps_yhats_dict['epoch']

    # Initialize the models
    models = []

    # Loop over the ensembles to build models for each
    assert(number_ensemble <= 10), 'whatcha think you got infinite come pew ters'
    for i in range(0, number_ensemble):
        # Bootstrap the rows so different models in the ensemble are less correlated
        # TODO REMOVE?
        train_df = sk.utils.resample(train_df, replace = True, 
            n_samples = int(train_df.shape[0]*bootstrap_pct),
            random_state = i)
        
        # Obtain the training input and output data and, if passed, validation data
        train_b_t_1 = train_df[b_t_1]
        train_x_t   = train_df[x_t]  
        train_y     = train_df[['r_tplus7']]
        if val_df is not None:
            val_b_t_1   = val_df[b_t_1]
            val_x_t     = val_df[x_t]  
            val_y       = val_df[['r_tplus7']]
        
        # According to which model in the ensemble it is, initialize parameters.
        # TODO REPLACE WITH RANDOMLY GRABBING INIT METHOD OUT OF LIST
        # -BUT SET SEED FOR RANDOM GRAB SO IT REPLICABLE
        if i==0:
            weight_initializer=initializers.HeNormal(seed=i)
            bias_initializer=initializers.GlorotUniform(seed=i)
        elif i==1:
            weight_initializer=initializers.GlorotUniform(seed=i)
            bias_initializer=initializers.RandomUniform(seed=i)
        elif i==2:
            weight_initializer=initializers.HeNormal(seed=i)
            bias_initializer=initializers.RandomUniform(seed=i)
        elif i==3:
            weight_initializer=initializers.RandomUniform(seed=i)
            bias_initializer=initializers.GlorotUniform(seed=i)
        elif i==4:
            weight_initializer=initializers.GlorotUniform(seed=i)
            bias_initializer=initializers.HeNormal(seed=i)
        else:
            weight_initializer=initializers.HeNormal(seed=i)
            bias_initializer=initializers.RandomUniform(seed=i)
        
        # Build the betas model from the time t covariates
        # TODO ADJUST SIZE OF NETWORK IN RELATON TO NUMBER OF COVARS
        # -THEY AHVE ~100 SO IF I HAVE LESS THEN DO PROPORTIONAL
        # -AND MAKE THEM POWERS OF 2
        model_b = tf.keras.models.Sequential()
        model_b.add(tf.keras.Input(shape=(len(b_t_1),)))
        model_b.add(Dense(32, activation='relu',
                          kernel_regularizer=regularizers.l1(l1=l1_penalty),
                          kernel_initializer=weight_initializer,
                          bias_initializer=bias_initializer))
        model_b.add(BatchNormalization())
        if number_hidden_layer >= 2:
            model_b.add(Dense(16, activation='relu',
                              kernel_regularizer=regularizers.l1(l1=l1_penalty),
                              kernel_initializer=weight_initializer,
                              bias_initializer=bias_initializer))
            model_b.add(BatchNormalization())
        if number_hidden_layer == 3:
            model_b.add(Dense(8, activation='relu',
                              kernel_regularizer=regularizers.l1(l1=l1_penalty),
                              kernel_initializer=weight_initializer,
                              bias_initializer=bias_initializer))
            model_b.add(BatchNormalization())
        model_b.add(Dense(number_factor, activation='linear',
                          kernel_initializer=weight_initializer,
                          bias_initializer=bias_initializer))

        # Form the x model from time t plus 1 returns
        model_x = tf.keras.models.Sequential()
        model_x.add(tf.keras.Input(shape=(len(x_t),)))
        model_x.add(Dense(number_factor, activation='linear',
                          kernel_initializer=weight_initializer,
                          bias_initializer=bias_initializer))

        # Form the dot product output for the combination of the two neurals
        mergedOut = Dot(axes=(1,1))([model_b.output, model_x.output])

        # Form the entire model
        model = Model([model_b.input, model_x.input], mergedOut)

        # Compile the model
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                      loss='mean_squared_error',
                      metrics=['mse'])

        # Prepare early stopping object 
        es = EarlyStopping(monitor='val_mse', mode='min', verbose=0, patience = 2) # VERBOSE 2
        
        # Fit the model
        # TODO CHANGE TO GPU!!
        with tf.device('/CPU:0'):
            if early_stopping == True:
                model.fit(x=[train_b_t_1, train_x_t], y=train_y, 
                          batch_size=batch_size,
                          validation_data=([val_b_t_1, val_x_t], val_y), 
                          epochs=epoch, verbose=0,
                          workers=4, callbacks=[es]) # VERBOSE 1
            else:
                model.fit(x=[train_b_t_1, train_x_t], y=train_y, 
                          batch_size=batch_size,
                          epochs=epoch, verbose=1,
                          workers=4)

        models.append(model)

    return models


In [8]:
def genYhats(df, models, oos_week, number_factor):
    # Obtain the covariates on x_t and b_t_1 sides of the network
    column_names = list(df.columns.values)
    column_names.remove('asset')
    column_names.remove('r_tplus7')
    column_names.remove('week_idx')
    covariates = column_names
    x_t   = [covar for covar in covariates if covar[:2] == 'x_']
    b_t_1 = [covar for covar in covariates if covar[:2] != 'x_']

    # Obtain the oos data
    oos_df = df[df.index == oos_week].copy()
    oos_x_t = oos_df[x_t]
    oos_b_t_1 = oos_df[b_t_1]

    # For each model form the beta hats
    b_hats = np.zeros((oos_df.shape[0],number_factor))
    for model in models:
        layer_name = model.layers[-3]._name 
        assert(model.layers[-3].output_shape[1] == number_factor)
        b_hat_layer = Model(inputs=model.input[0],
                            outputs=model.get_layer(layer_name).output)
        b_hat = b_hat_layer.predict(oos_b_t_1)
        b_hats += b_hat
        
    # Obtain the average b_hat across the models
    b_hats = b_hats/len(models)
    
    # output statistics- 5 stat summary of Beta Hat
    # TODO REPLACE THIS WITH A FUNC TO DO THIS
#     b_hats_min = b_hats.min()
#     b_hats_max = b_hats.max()
#     b_hats_25_percentile = np.percentile(b_hats,25)
#     b_hats_75_percentile = np.percentile(b_hats,75)
#     b_hats_mean = b_hats.mean()
#     print('Min beta_hats: %.3f'% b_hats_min)
#     print('Q1 beta_hats: %.3f'% b_hats_25_percentile)
#     print('Mean beta_hats: %.3f'% b_hats_mean)
#     print('Q3 beta_hats: %.3f'% b_hats_75_percentile)
#     print('Max beta_hats: %.3f'% b_hats_max)
#     print('\n')

    # Form the sample average of the estimated factors up to the OOS week
    weeks = np.unique(df[df.index < oos_week].index)
    lambda_hats = np.zeros(number_factor)
    for prev_week in weeks:
        prev_x_t = df[df.index == prev_week][x_t]
        f_hats = np.zeros(number_factor)
        for model in models:
            layer_name = model.layers[-2]._name 
            assert(model.layers[-2].output_shape[1] == number_factor)
            f_hat_layer = Model(inputs=model.input[1],
                                outputs=model.get_layer(layer_name).output)
            f_hat = f_hat_layer.predict(prev_x_t)
            assert(all(np.isclose(f_hat[0,:], f_hat[1,:])))
            f_hats += f_hat[0,:]
        f_hats = f_hats / len(models)
        lambda_hats += f_hats
        lambda_hats = lambda_hats / len(weeks)
        
    # summary of lambda_hats - 5 stat summary
    # TODO REPLACE THIS WITH A FUNC TO DO THIS
#     lambda_hats_min = lambda_hats.min()
#     lambda_hats_max = lambda_hats.max()
#     lambda_hats_25_percentile = np.percentile(lambda_hats,25)
#     lambda_hats_75_percentile = np.percentile(lambda_hats,75)
#     lambda_hats_mean = lambda_hats.mean()
#     print('Min lambda_hats: %.3f'% lambda_hats_min)
#     print('Q1 lambda_hats: %.3f'% lambda_hats_25_percentile)
#     print('Mean lambda_hats: %.3f'% lambda_hats_mean)
#     print('Q3 lambda_hats: %.3f'% lambda_hats_75_percentile)
#     print('Max lambda_hats: %.3f'% lambda_hats_max) 
#     print('\n')
        
    # Form yhats
    yhats = np.matmul(b_hats, lambda_hats)
    
    # output stats of yhats - 5 stat summar
    # TODO REPLACE THIS WITH A FUNC TO DO THISy 
#     y_hats_min = yhats.min()
#     y_hats_max = yhats.max()
#     y_hats_25_percentile = np.percentile(yhats,25)
#     y_hats_75_percentile = np.percentile(yhats,75)
#     y_hats_mean = yhats.mean()
#     print('Min y_hats: %.3f'% y_hats_min)
#     print('Q1 y_hats: %.3f'% y_hats_25_percentile)
#     print('Mean y_hats: %.3f'% y_hats_mean)
#     print('Q3 y_hats: %.3f'% y_hats_75_percentile)
#     print('Max y_hats: %.3f'% y_hats_max)
#     print('\n')
    

    return yhats

In [9]:
def runCV(df, asset_universe_dict, last_train_year=2018, val_end_year=2020):
    # Initialize hp result objects
    hps_yhats_dict_list = []
    hps_mse_df_list     = []
    
    # Initialize the hyperparameter grid  
    # TODO MOVE THIS OUTSIDE SO I PASS THIS IN
    number_hidden_layers = [3, 2, 1]
    number_factors       = [6, 5, 4, 3, 2, 1]
    learning_rates       = [1e-5, 1e-3, 1e-1]
    batch_sizes          = [128] # note: ensure powers of 2 for eff, [64,256]
    l1_penalties         = [1, 1e-1, 1e-2, 1e-3]
    number_ensembles     = [10] 
    early_stopping       = [True]
    bootstrap_pcts       = [1]
    epochs               = [10] 

    # Determine the weeks in the validation window  
    val_weeks = np.unique(df[(df.index.year > last_train_year)  
                             & (df.index.year <= val_end_year)].index.values)  

    # Generate yhats for every hyperparameter grid point
    # TODO ADJUST TO WHATEVER THE KEYS ARE IN THE HP DICT
    # -use those keys to form the dict inside or maybe just get like the indices
    for hps in itertools.product(number_hidden_layers,
                                 number_factors,
                                 learning_rates,
                                 batch_sizes,
                                 l1_penalties,
                                 number_ensembles,
                                 early_stopping,
                                 bootstrap_pcts,
                                 epochs):
        hps_yhats_dict = {'number_hidden_layer': hps[0],
                          'number_factor': hps[1],
                          'learning_rate': hps[2],
                          'batch_size': hps[3],
                          'l1_penalty': hps[4],
                          'number_ensemble': hps[5],
                          'early_stopping': hps[6],
                          'bootstrap_pct': hps[7],
                          'epoch': hps[8],
                          'yhats': np.array([]),
                          'ys':    np.array([])}
        print(hps_yhats_dict, '\n')
        tic = time.perf_counter()
        # TODO DO SOME LOGIC TO FIGURE OUT THE WINDOWS TO FIT A MODEL FOR SO I LOOP OVER THOSE INITIAL DATES
        # -fit teh model once
        # -then predict on all subsequent weeks
        # -maybe one month at time?
        # -maybe one week at a time?
        # -let's try once a month to see how it does
        for val_week in val_weeks:
#             print(val_week, '\n')
            temp_df = df[df.index <= val_week].copy()
            temp_df = subsetToAssetUniversePerWeek(temp_df, asset_universe_dict, oos_week=val_week)
            train_df = temp_df[temp_df.index < val_week].copy()
            val_df   = temp_df[temp_df.index == val_week].copy()
            
            models = fitAutoencoder(train_df, hps_yhats_dict, val_df=val_df, 
                                    early_stopping=hps_yhats_dict['early_stopping'])
            yhats  = genYhats(temp_df, models, val_week, hps_yhats_dict['number_factor'])
            ys     = val_df.r_tplus7.values

            hps_yhats_dict['yhats'] = np.append(hps_yhats_dict['yhats'], yhats)
            hps_yhats_dict['ys']    = np.append(hps_yhats_dict['ys'], ys)

            val_ys_todate = hps_yhats_dict['ys']
            rw_val_mse    = np.mean(np.square(val_ys_todate))
            model_val_mse = np.mean(np.square(val_ys_todate - hps_yhats_dict['yhats']))
#             print('\n val random walk mse: ' + str(rw_val_mse))
#             print('\n val model mse: ' + str(model_val_mse))
#             print('\n val model mse winning?: ' + str(model_val_mse < rw_val_mse))
#             print('\n\n')
# TODO WRITE FUNC HERE TO REPORT SIMPLE STUFF LIKE DIST OF PREDICTED RETURNS AND PREDICTIVE R^2
        # Save run time and space out result print out
        toc = time.perf_counter()
#         print('\n\n\n')

        # Update lists of results
        # TODO WRITE FUNC TO REPORT MORE STUFF HERE FROM OTHER NOTEBOOKS
        # -including all the results that i will have for test period
        hps_yhats_dict_list.append(hps_yhats_dict)
        cv_results_dict = hps_yhats_dict.copy()
        del cv_results_dict['yhats']
        del cv_results_dict['ys']
        cv_results_dict['runtime_mins'] = round((toc - tic)/60, 0)
        cv_results_dict['mse'] = model_val_mse
        hps_mse_df_list.append(pd.DataFrame(cv_results_dict, index=[0]))

        # Save CV results
        cv_df = pd.concat(hps_mse_df_list, ignore_index=True)
        timestr = time.strftime("%Y%m%d_%H%M%S")
        fp = '../4-output/cv-results-autoencoder-' + timestr + '.csv'
        cv_df.to_csv(fp, index=False)

    return hps_yhats_dict_list


In [ ]:
def runCV(cv_df: pd.DataFrame, 
    lhs_col: str,
    val_start_date: str,
    num_cpus: int,
    arch_name: str, 
    out_fp: str,
    periods_in_year: int) -> list:
    ''' run step-forward cross validation to select optimal hyperparameters for target model 
        returning fitting yhats and models as well as outputting results to csv.

    Args:
        cv_df (pd.DataFrame): panel of training and val data with date index, LHS variable named 
                              `lhs_col`, and remaining cols are RHS features.
        lhs_col (str): name of the lhs target column.
        val_start_date (str): the first date for the validation period.
        num_cpus (int): number of cpus to use when parallelizing.
        arch_name (str): name of architecture to use when saving intermitent results.
        out_fp (str): filepath to output the csv file without `.csv' on end.
        periods_in_year (int): how many rows of the data constitute a year.
          
    Returns:
        (list): list of dictionaries for each hyperparameter fit where each list contains keys of:
                    `hps` is dict of hyperparameter combination,
                    `yhats` is an array of validation period yhats, and,
                    `models` is list of fitted models.  
    '''
    # initialize args
    val_dates = np.unique(cv_df[val_start_date:].index.values)
    results_list = []
    csv_dict_list = []
    rhs_cols = list(cv_df.columns.values)
    rhs_cols.remove(lhs_col)

    # initialize hp grid
    # GO SCOPE GU EMP AP VIA ML HP'S AND JUST DO THAT
    # SCOPE GU AUTOENCODER PAPER TO ENSURE NO OTHER HP'S MENTIONED
    num_estimators  = [10, 20, 40] # 20 seems good but do fine grid 10, 20, 30, 40, 50, 100 later
    subsamples      = [0.7, 0.8, 0.9] # 0.8 seems good but can do fine grid 0.75 to 0.95 later
    min_samp_leafs  = [64, 128, 256, 512, 1024]  # 1024 seems good can but can do fine grid 512 to 1536 later
    max_depths      = [1, 2] # keep trying 1,2 
    max_features    = [0.005, 0.01, 0.02, 0.05, 0.1, 0.2] # kinda unclear; do fine grid 0.005 to 0.3 later
    loss_funcs      = ['squared_error']
    weight_funcs    = ['exp', 'linear', 'y']  # y seems epic    

    # loop over all hp combos
    for hps in itertools.product(num_estimators,
                                 subsamples,
                                 min_samp_leafs,
                                 max_depths, 
                                 max_features,
                                 loss_funcs,
                                 weight_funcs):
        # initialize this hp iter args
        results_dict = {}
        results_dict['hps'] = {'num_estimator': hps[0],
            'subsample': hps[1],
            'min_samp_leaf': hps[2],
            'max_depth': hps[3],
            'max_feature': hps[4],
            'loss_func': hps[5],
            'weight_func': hps[6]}

        # monitor progress
        print(results_dict['hps'], '\n') 

        # fit model on all val dates
        tic = time.perf_counter()
        def loopOverValDates(val_date): # set up as a func to loop over
            # form train and val data
            train_df = cv_df[cv_df.index < val_date].copy()
            val_df   = cv_df[cv_df.index == val_date].copy()

            # fit model and generate yhats for val week
            model, train_r2_pred, train_yhats = fitRF(train_df, lhs_col, rhs_cols, results_dict['hps'])
            yhat = genRFYhats(val_df, model, rhs_cols)

            return yhat, model, train_r2_pred, train_yhats

        val_results = Parallel(n_jobs=int(num_cpus/2))(delayed(loopOverValDates)(val_date) for val_date in tqdm(val_dates))

        # extract validation periods results
        yhats_list = []
        models_list = []
        train_r2_pred_list = []
        train_yhats_list = []
        for t in range(len(val_results)):
            yhats_list.append(val_results[t][0])
            models_list.append(val_results[t][1])
            train_r2_pred_list.append(val_results[t][2])
            train_yhats_list.append(val_results[t][3])
        yhats = np.array(yhats_list)
        positions = np.sign(yhats)
        ys    = cv_df[val_start_date:][lhs_col].values
        results_dict['yhats'] = yhats

        # save results to master result list    
        results_list.append(results_dict)

        # save results to csv to monitor during cv
        toc = time.perf_counter()
        csv_dict = results_dict['hps'].copy()
        del results_dict
        csv_dict['arch_name'] = arch_name
        csv_dict['val_start_date'] = val_start_date
        csv_dict['val_end_date'] = np.datetime_as_string(np.max(cv_df.index.values))[:10]
        csv_dict['runtime_mins'] = round((toc - tic)/60, 0) 
        csv_dict['mean_r2_pred_train'] = np.mean(np.array(train_r2_pred_list))
        csv_dict['r2_pred_val'] = 1-np.mean(np.square(ys-yhats))/np.mean(np.square(ys))
        tc = calcTransactionCosts(positions)
        returns = positions*ys - tc
        csv_dict['geom_mean_1h'] = calcGeomAvg(returns)
        csv_dict['sharpe_ann'] = calcSharpe(returns, periods_in_year=periods_in_year)
        csv_dict['sd_ann'] = calcSD(returns, annualized=True, periods_in_year=periods_in_year)
        csv_dict['ts_mean_ann'] = calcTSAvgReturn(returns, annualized=True, periods_in_year=periods_in_year)
        for lev in [2, 3, 4]:
            lev_pos = positions*lev
            lev_tc = calcTransactionCosts(lev_pos)
            lev_returns = lev_pos*ys - lev_tc 
            csv_dict['geom_mean_x'+str(lev)] = calcGeomAvg(lev_returns)
            csv_dict['sharpe_x'+str(lev)] = calcSharpe(lev_returns, periods_in_year=periods_in_year)
            csv_dict['sd_ann_x'+str(lev)] = calcSD(lev_returns, annualized=True, periods_in_year=periods_in_year)
            csv_dict['ts_mean_ann_x'+str(lev)] = calcTSAvgReturn(lev_returns, annualized=True, periods_in_year=periods_in_year)
        csv_dict_list.append(csv_dict)
        results_df = pd.DataFrame(csv_dict_list)

        timestr = time.strftime("%Y%m%d_%H%M%S")
        fp = out_fp+'_'+arch_name+'_'+timestr+'.csv'
        results_df.to_csv(fp, index=False)

        # output results to track
        print(csv_dict['runtime_mins'])
        print(f"val r2_pred: {csv_dict['r2_pred_val']:.4f}")
        print(f"val sharpe: {csv_dict['sharpe_ann']:.4f}")

        print('\n\n')
        gc.collect()

    return results_list

In [ ]:
def plotYvsYhat(y: np.ndarray, yhats: np.ndarray, num_quantiles: int = 10) -> None:
    """ Plot the average values of y (returns) against bins of yhat (predicted returns).

    Args:
        y (np.ndarray): A vector of target variable.
        yhats (np.ndarray): A vector of fitted values.
        num_quantiles (int): The number of quantiles to bin yhats into. Default is 10.
    """
    # Create a DataFrame with y and yhats
    temp_df = pd.DataFrame({'y': y, 'yhat': yhats})

    # Bin yhats into quantiles
    temp_df['yhat'] = pd.qcut(temp_df['yhat'], q=num_quantiles, labels=False)

    # Group y by yhat and plot the means
    temp_df.groupby('yhat')['y'].mean().plot()

    # Set plot title
    plt.title('Mean Return vs. Quantiles of Predicted Returns')

    # Show the plot
    plt.show()


In [10]:
def GenTestYhats(df, opt_hps, test_year=2021): 
    test_weeks = np.unique(df[df.index.year == test_year].index.values)
    
    test_df = pd.DataFrame()

    for test_week in test_weeks:
        # TODO DO SOME LOGIC TO FIGURE OUT THE WINDOWS TO FIT A MODEL FOR SO I LOOP OVER THOSE INITIAL DATES
        # -fit teh model once
        # -then predict on all subsequent weeks
        # -maybe one month at time?
        # -maybe one week at a time?
        # -let's try once a month to see how it does
        print(test_week, '\n')
        temp_df  = df[df.index <= test_week].copy()
        temp_df  = subsetToAssetUniversePerWeek(temp_df, asset_universe_dict, oos_week=test_week)
        train_df = temp_df[temp_df.index < test_week].copy()
        oos_df   = temp_df[temp_df.index == test_week].copy()
        
        models   = fitAutoencoder(train_df, opt_hps, val_df=None, early_stopping=False)
        yhats    = genYhats(oos_df, models, test_week, opt_hps['number_factor'])
       
        oos_df = oos_df[['asset', 'r_tplus7']]
        oos_df['yhat'] = yhats
        test_df = pd.concat((test_df, oos_df))
        rw_mse = np.mean(np.square(test_df.r_tplus7.values))
        model_mse = np.mean(np.square(test_df.r_tplus7.values - test_df.yhat.values))
        print('\n test random walk mse: ' + str(rw_mse))
        print('\n test model mse: ' + str(model_mse))
        print('winning?: ' + str(model_mse < rw_mse))
        print('\n')
    
    return test_df


In [ ]:
def predictTestPeriod(df: pd.DataFrame,
    lhs_col: str,
    test_start_date: str,
    test_end_date: str,
    opt_model_dict: dict,
    num_cpus: int) -> np.array:
    ''' predict for test period using backtested model.

    Args:
        df (pd.DataFrame): panel of training and test data with date index, LHS variable named 
                           `lhs_col`, and remaining cols are RHS features.
        lhs_col (str): name of the lhs target column.
        test_start_date (str): the first date for the test period.
        test_end_date (str): the last date for the test period.
        opt_model_dict (dict): hyperparameters for optimal backtested model.
        num_cpus (int): number of cpus to use when parallelizing.
    
    Returns:
        yhats (np.array): vector of yhat positions for entire test period.
    '''
    # form list of rhs col
    rhs_cols = list(df.columns.values)
    rhs_cols.remove(lhs_col)

    # form lhs for oos df
    oos_df = df.copy()
    ret_threshold = opt_model_dict['ret_threshold']
    oos_df['y'] = 1
    oos_df.loc[oos_df[lhs_col] > ret_threshold, 'y'] = 2
    oos_df.loc[oos_df[lhs_col] < -ret_threshold, 'y'] = 0

    # form test dates to loop over
    test_dates = np.unique(oos_df[test_start_date:test_end_date].index.values)

    def loopOverTestDates(test_date):
        # form train and val data
        temp_df   = oos_df[oos_df.index <= test_date].copy()
        train_df  = temp_df[temp_df.index < test_date].copy()
        train_y_btc_eth_diff = train_df[lhs_col].values
        train_df  = train_df.drop(lhs_col, axis=1)
        test_df   = temp_df[temp_df.index == test_date].copy()
        test_df   = test_df.drop(lhs_col, axis=1)

        # fit and predict
        model, train_acc, train_yhats = fitRF(train_df, 'y', rhs_cols, 
                                            hps=opt_model_dict, 
                                            ys_real=train_y_btc_eth_diff)
        yhats = genRFYhats(test_df, model, 'y', rhs_cols)

        return yhats, model, train_acc, train_yhats

    # predict for entire test period
    test_results = Parallel(n_jobs=int(num_cpus/4))(delayed(loopOverTestDates)(test_date) for test_date in tqdm(test_dates))

    # extract test yhats to return
    yhats_list = []
    for t in range(len(test_results)):
        yhats_list.append(test_results[t][0])
    yhats = np.array(yhats_list)-1

    return yhats

In [ ]:
def reportTestPeriodResults(yhats: np.array, y_real: np.array):
    ''' report various portfolio statistics for the test period.
    
    Args:
        yhats (np.array): vector of actual portfolio positions.
        y_real (np.array): vector of real return difference for target instrument.
    '''
    # calculate returns post transaction costs
    tc = calcTransactionCosts(yhats)
    returns = yhats*y_real - tc
    
    # calc portfolio statistics
    geom_mean = calcGeomAvg(returns)
    sharpe    = calcSharpe(returns, periods_in_year=365*12)
    max_dd    = calcMaxDrawdown(returns)
    max_1week = calcMaxOneWeekLoss(returns, periods_in_week=7*12)

    # report
    print(f"Geometric average 2 hour return: {geom_mean:.6f}")
    print(f"Annualized Sharpe: {sharpe:.3f}")
    print(f"Maximum drawdown: {max_dd:.4f}")
    print(f"Maximum loss over any one week period: {max_1week:.4f}")

    # plot classification
    plotYvsYhat(y=y_real, yhats=yhats)

In [ ]:
# TODO RUN UNIFACTOR IMPORTANCE ON CHARACTERISTICS TO CHOOSE 20-100 PENDING WHEN I GET ENOUGH ASSETS IN THE PANEL.
# -FOCUS ON RAW STUFF FOR NOW AS I WANT TO SHOW TRANSFORMER LEARNS TEMPORAL RELATIONSHIPS
# --CAN GIVE IT THE FANCY CHARACTERISTICS IN SECOND RUN

In [ ]:
if __name__ == "__main__":
    # set args
    IN_FP           = '../../data/clean/btcusdt-1h-eng_feats.pkl'
    CV_OUT_FP       = '../output/cv_results'
    LHS_COL         = 'r_tplus1'
    VAL_START_DATE  = '2023-01-01'
    VAL_END_DATE    = '2023-02-28'
    TEST_START_DATE = '2023-03-01'
    NUM_CPUS        = 2 # TODO change to 22 when doing on desktop
    PERIODS_IN_YEAR = 365*24

    # read in data
    df = pd.read_pickle(IN_FP)
    df = df.set_index('timestamp')
    df = df.astype('float32')

    # cv rf
    VAL_START_DATE = '2023-01-01'
    VAL_END_DATE = '2023-01-14'
    results_rf_list = runCV(df[:VAL_END_DATE], LHS_COL, VAL_START_DATE, 
                            NUM_CPUS, 'rf_reg_1hour', CV_OUT_FP, PERIODS_IN_YEAR)



In [ ]:
# TODO Update CV method to strip out everything that will be common across CV's to make them functions: fitting on all val dates, gen CV results, others?
# TODO ADD function to report max dd and add it to the CV results func to have in the csv
# TODO add to csv to report 5 stat summary of yhats
# TODO add to csv to report num of position changes


In [ ]:
# TODO pull helper functions out of first cell in backtest notebook to create a basic class with GPT's help of quant tools that i put in some folder on my computer of my python methods
# maybe move into a folder where i keep my python tools
# make it a class so i import as needed
# create a repo for that
# ask chatgpt how to structure it in various python files
# maybe ask for a template folder and file structure for the library
# add a pointer to it for the automa capital stuff
# TODO Move helper functions to a call in utils file.


In [ ]:
#if __name__ == "__main__":
# load in the data
input_fp = '../3-data/clean/panel_train.pkl'
df = pd.read_pickle(input_fp)
df = df.set_index('date')
df = df.sort_values(by=['date', 'asset'])
with open('../3-data/clean/asset_universe_dates_and_lists.pkl', 'rb') as handle:
    asset_universe_dict = pickle.load(handle)

# drop rows and columns such that data will work for conditional autoencoder (CA)
df = dropRowsAndColsForCA(df)

# form the char-sorted portfolios for factor side of CA input
df = formPortfolioReturnCovariates(df)

# sort the data
df = df.sort_values(by=['date', 'asset'])

In [ ]:
# TODO PASS IN FUNC FOR FITTING MODEL
# TODO PASS IN FUNC FOR PREDICTING WITH MODEL
# TODO PASS IN PARM DIC FOR BOTH FIT AND PREDICT SO I CAN JUST UPDATE TEH DYNAMIC PARAMS WHILE REST ARE STATIC

In [ ]:
# run CV
hps_yhats_list = runCV(df, asset_universe_dict, last_train_year = 2018, val_end_year=2020)

In [ ]:
# TODO: add code to select the best hps combo from the above list
#       i just ran manually with setting the best CV point

In [ ]:
# form validation period results
df = df[['asset', 'week_idx', 'r_tplus7']]
df = subsetToAssetUniverseFull(df, asset_universe_dict, train_or_test='train')
df = df[df.index.year >= 2019]
df['yhat'] = hps_yhats_list[0]['yhats']
df      = labelPortfolioWeights(df)
annl_tc = calcAnnualTransactionCosts(df)
df['r'] = df.prtfl_wght*df.r_tplus7
r_df    = df.groupby(['date'])[['r']].sum()
annl_ret = calcPortfolioReturn(r_df)
annl_sharpe = calcPortfolioSharpe(r_df)
max_dd = max_draw_down(r_df)
max_1mo_loss = max_1_month_loss(r_df)
print('annual transaction costs in simple return terms: ' + str(np.round(annl_tc, 4)))
print('annual simple return before trans costs: ' + str(np.round(annl_ret, 4)))
print('annual sharpe: '+str(np.round(annl_sharpe, 2)))
print('max drawdown : '+str(np.round(max_dd, 2)))
print('max one month loss : '+str(np.round(max_1mo_loss, 2)))

In [ ]:
# TODO GO SCOPE OLD NOTEBOOK FOR LATEST NUMBERS

In [ ]:
# import and clean up data for test fitting
input_fp = '../3-data/clean/panel_oos.pkl'
df = pd.read_pickle(input_fp)
df = df.set_index('date')
df = df.sort_values(by=['date', 'asset'])
opt_hps = hps_yhats_list[0].copy()
del opt_hps['yhats']
del opt_hps['ys']
test_df = GenTestYhats(df, opt_hps) 
test_df = test_df.merge(df[['asset', 'week_idx']],
                        on=['date', 'asset'],
                        how='inner', 
                        validate='one_to_one')

# form test period results
test_df = labelPortfolioWeights(test_df)
annl_tc = calcAnnualTransactionCosts(test_df)
test_df['r'] = test_df.prtfl_wght*test_df.r_tplus7
r_df    = test_df.groupby(['date'])[['r']].sum()
annl_ret = calcPortfolioReturn(r_df)
annl_sharpe = calcPortfolioSharpe(r_df)
max_dd = max_draw_down(r_df)
max_1mo_loss = max_1_month_loss(r_df)
print('annual transaction costs in simple return terms: ' + str(np.round(annl_tc, 4)))
print('annual simple return before trans costs: ' + str(np.round(annl_ret, 4)))
print('annual sharpe: '+str(np.round(annl_sharpe, 2)))
print('max drawdown : '+str(np.round(max_dd, 2)))
print('max one month loss : '+str(np.round(max_1mo_loss, 2)))

# TODO GO SCOPE OLD NOTEBOOK FOR LATEST NUMBERS


In [ ]:
# TODO ALSO NEED TO TRY A MODEL WHERE WE SET MISSING ASSETS LHS 
# AND RHS TO -2 AND THEN WE HAVE BALANCED PANEL SO WE FIT ON THE 
# MATRIX OF RHS ACROSS ASSETS AS OPPOSED OT INDIVIDUAL DATE-ASSET 
# AND THEN THE NETWORK LEARNS ASSET-SPECIFIC PARAMETERS

In [ ]:
# TODO THIS NOTEBOOK ENDS WITH SAVING A DF OF DATE, ASSET, PREDICTED RETURN FOR A VAL OR TEST PERIOD

In [ ]:
# TODO: Figure out why all the test period yhats are close to zero
#       use the comments to report out the distribution 
#       to figure out if factor side or beta side that is turning all to zero